# СДЗ 4

## Задача 1. Подмножества (2 балла)

Дан целочисленный массив уникальных элементов. Нужно вывести все возможные подмножества (the power set).

надо сгенерировать все подмножества

решил через backtracking - на каждом шаге либо добавляем текущий элемент в подмножество либо нет

начинаем с пустого списка, потом рекурсивно перебираем элементы. важный момент - нужно копировать список через [:], иначе все ссылки будут на один объект

по сложности получается O(n * 2^n) потому что 2^n подмножеств и на каждое тратим O(n) на копирование

In [5]:
def getSubsets(nums):
    result = []
    
    def backtrack(start, current):
        result.append(current[:])
        
        for i in range(start, len(nums)):
            current.append(nums[i])
            backtrack(i + 1, current)
            current.pop()
    
    backtrack(0, [])
    return result

print(getSubsets([1, 2, 3]))
print(getSubsets([0]))
print(getSubsets([5, 1, 8]))

[[], [1], [1, 2], [1, 2, 3], [1, 3], [2], [2, 3], [3]]
[[], [0]]
[[], [5], [5, 1], [5, 1, 8], [5, 8], [1], [1, 8], [8]]


Теперь изначальный массив может содержать дубликаты. Остальная задача осталась прежней.

сортируем массив, а потом при переборе пропускаем одинаковые элементы на одном уровне рекурсии

проверка `i > start and nums[i] == nums[i-1]` как раз отсекает дубли

асимптотика та же O(n * 2^n) в худшем случае, может быть меньше из-за пропуска дубликатов

In [6]:
def getSubsetsWithDuplicates(nums):
    nums.sort()
    result = []
    
    def backtrack(start, current):
        result.append(current[:])
        
        for i in range(start, len(nums)):
            if i > start and nums[i] == nums[i - 1]:
                continue
            
            current.append(nums[i])
            backtrack(i + 1, current)
            current.pop()
    
    backtrack(0, [])
    return result

print(getSubsetsWithDuplicates([1, 2, 2]))
print(getSubsetsWithDuplicates([0]))
print(getSubsetsWithDuplicates([4, 4, 4, 1, 4]))

[[], [1], [1, 2], [1, 2, 2], [2], [2, 2]]
[[], [0]]
[[], [1], [1, 4], [1, 4, 4], [1, 4, 4, 4], [1, 4, 4, 4, 4], [4], [4, 4], [4, 4, 4], [4, 4, 4, 4]]


## Задача 2. Сумма (2 балла)

задача на поиск комбинаций с заданной суммой

тоже через backtracking - на каждом шаге пробуем добавить число и рекурсивно ищем дальше

можно использовать одно число несколько раз, поэтому в рекурсии передаем индекс i (а не i+1)

останавливаемся когда remaining == 0 (нашли комбинацию) или remaining < 0 (перебор)

сложность экспоненциальная, зависит от target и минимального элемента в candidates. грубо O(target/min * 2^(target/min))

In [7]:
def findCombinationSum(candidates, target):
    result = []
    
    def backtrack(start, current, remaining):
        if remaining == 0:
            result.append(current[:])
            return
        
        if remaining < 0:
            return
        
        for i in range(start, len(candidates)):
            current.append(candidates[i])
            backtrack(i, current, remaining - candidates[i])
            current.pop()
    
    backtrack(0, [], target)
    return result

print(findCombinationSum([2, 3, 6, 7], 7))
print(findCombinationSum([2, 3, 5], 8))
print(findCombinationSum([2], 1))

[[2, 2, 3], [7]]
[[2, 2, 2, 2], [2, 3, 3], [3, 5]]
[]


Та же задача, но каждое число из числа кандидатов может использоваться в комбинации только один раз.

меняем i на i+1 в рекурсивном вызове

сортируем массив и делаем break когда текущий элемент больше remaining. если отсортировано, то все следующие элементы тоже будут больше, смысла продолжать нет

также нужно пропускать дубликаты чтобы не генерировать одинаковые комбинации

в худшем случае O(2^n) где n - количество кандидатов

In [8]:
def findCombinationSumUnique(candidates, target):
    candidates.sort()
    result = []

    def backtrack(start, current, remaining):
        if remaining == 0:
            result.append(current[:])
            return

        for i in range(start, len(candidates)):
            if i > start and candidates[i] == candidates[i - 1]:
                continue
            if candidates[i] > remaining:
                break

            current.append(candidates[i])
            backtrack(i + 1, current, remaining - candidates[i])
            current.pop()

    backtrack(0, [], target)
    return result

print(findCombinationSumUnique([10, 1, 2, 7, 6, 1, 5], 8))
print(findCombinationSumUnique([2, 5, 2, 1, 2], 5))
print(findCombinationSumUnique([1, 1, 1], 2))

[[1, 1, 6], [1, 2, 5], [1, 7], [2, 6]]
[[1, 2, 2], [5]]
[[1, 1]]


## Задача 3. Королевы (2 балла)

Требуется разместить n ферзей на шахматной доске размером $n \times n$ таким образом, чтобы никакие два ферзя не атаковали друг друга.

Задано целое число $n$. Найдите все различные решения задачи о $n$ ферзях. Вы можете вернуть ответ в любом порядке.
Каждое решение содержит различную конфигурацию доски с размещением $n$ ферзей, где «Q» и «.» обозначают ферзя и пустое место соответственно.

размещаем по одному ферзю в каждой строке

для каждой строки перебираем все колонки и проверяем можно ли поставить ферзя:
- по вертикали - смотрим нет ли ферзя в этой колонке выше
- по диагоналям - для левой диагонали row-col константа, для правой row+col константа

использую setы для быстрой проверки занятых колонок и диагоналей

когда разместили всех n ферзей, формируем строковое представление и добавляем в result

сложность O(n!)

In [9]:
def solveNQueens(n):
    result = []
    board = [['.'] * n for _ in range(n)]
    
    cols = set()
    diag1 = set()
    diag2 = set()
    
    def isSafe(row, col):
        return col not in cols and (row - col) not in diag1 and (row + col) not in diag2
    
    def placeQueen(row, col):
        board[row][col] = 'Q'
        cols.add(col)
        diag1.add(row - col)
        diag2.add(row + col)
    
    def removeQueen(row, col):
        board[row][col] = '.'
        cols.remove(col)
        diag1.remove(row - col)
        diag2.remove(row + col)
    
    def backtrack(row):
        if row == n:
            result.append([''.join(r) for r in board])
            return
        
        for col in range(n):
            if isSafe(row, col):
                placeQueen(row, col)
                backtrack(row + 1)
                removeQueen(row, col)
    
    backtrack(0)
    return result

solutions = solveNQueens(4)
for sol in solutions:
    for row in sol:
        print(row)
    print()

print(f"n=4: {len(solutions)}")
print(f"n=8: {len(solveNQueens(8))}")

.Q..
...Q
Q...
..Q.

..Q.
Q...
...Q
.Q..

n=4: 2
n=8: 92
